In [1]:
import wandb
from tqdm import tqdm
import numpy as np

api = wandb.Api()

def load_impact_of_lmbda_runs(acq):
    runs = api.runs(path="ziv-scully-group/PandoraBayesOPT", filters={
        "sweep": "xcn0phan",
        "config.amplitude": 1,
        "config.dim": 8,
        "config.kernel": "Matern52",
        "config.lengthscale": 0.1,  
        "config.policy": acq})
    
    configs_and_metrics = []
    for run in tqdm(runs):
        metric_keys = ["cumulative cost","best observed","lmbda"]
        history = run.scan_history(keys = metric_keys, page_size=1_000_000_000)
        metrics = {k: [d[k] for d in history] for k in metric_keys}
        summary_metric_keys = ["global optimum value"]
        summary_history = run.scan_history(keys = summary_metric_keys, page_size=1_000_000_000)
        metrics.update({k: [d[k] for d in summary_history] for k in summary_metric_keys})
        configs_and_metrics.append((run.config, metrics))

    return configs_and_metrics

In [2]:
acquisition_functions = [
    'Gittins_Lmbda_1_Step_Divide2',
    'Gittins_Lmbda_1_Step_Divide10',
    'Gittins_Lmbda_1_Step_Divide100',
    'Gittins_Lmbda_0001_Step_Divide2',
    'Gittins_Lmbda_0001_Step_Divide10',
    'Gittins_Lmbda_0001_Step_Divide100'
]

In [4]:
grouped_runs = {(a): load_impact_of_lmbda_runs(a) for a in acquisition_functions}

100%|██████████| 256/256 [31:03<00:00,  7.28s/it]   


In [5]:
for a in acquisition_functions:
    config_and_metrics_per_seed = grouped_runs[a]

    cumulative_cost_per_seed = np.array([m['cumulative cost'] for (_,m) in config_and_metrics_per_seed]).T
    best_observed_per_seed = np.array([m['best observed'] for (_,m) in config_and_metrics_per_seed]).T
    # Handling potential empty arrays
    if cumulative_cost_per_seed.size == 0 or best_observed_per_seed.size == 0:
        continue  # Skip this iteration if there's no data
    global_optimum_per_seed = np.array([m['global optimum value'][0] for (_,m) in config_and_metrics_per_seed if len(m['cumulative cost'])>0 and len(m['best observed'])>0])

    regret_per_seed = global_optimum_per_seed - best_observed_per_seed
    print(a, regret_per_seed.shape)

    regret_25 = np.quantile(regret_per_seed, 0.25, axis=1)
    regret_50 = np.quantile(regret_per_seed, 0.5, axis=1)
    regret_75 = np.quantile(regret_per_seed, 0.75, axis=1)

    output = np.stack((cumulative_cost_per_seed.mean(axis=1), regret_25, regret_50, regret_75),axis=-1)

    np.savetxt(f"results/gittins_step_divide_256/{a}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

Gittins_Lmbda_1_Step_Divide2 (200, 256)
Gittins_Lmbda_1_Step_Divide10 (200, 256)
Gittins_Lmbda_1_Step_Divide100 (200, 256)
Gittins_Lmbda_0001_Step_Divide2 (200, 256)
Gittins_Lmbda_0001_Step_Divide10 (200, 256)
Gittins_Lmbda_0001_Step_Divide100 (200, 256)


In [6]:
for a in acquisition_functions:
    config_and_metrics_per_seed = grouped_runs[a]

    lmbda_per_seed = np.array([m['lmbda'] for (_,m) in config_and_metrics_per_seed]).T
    # Handling potential empty arrays
    if lmbda_per_seed.size == 0:
        continue  # Skip this iteration if there's no data

    print(a, lmbda_per_seed.shape)

    lmbda_25 = np.quantile(lmbda_per_seed, 0.25, axis=1)
    lmbda_50 = np.quantile(lmbda_per_seed, 0.5, axis=1)
    lmbda_75 = np.quantile(lmbda_per_seed, 0.75, axis=1)

    output = np.stack((cumulative_cost_per_seed.mean(axis=1), lmbda_25, lmbda_50, lmbda_75),axis=-1)

    np.savetxt(f"results/lmbda_history_256/{a}.csv", output, header="cc, q25, q50, q75", delimiter=', ', comments='')

Gittins_Lmbda_1_Step_Divide2 (200, 256)
Gittins_Lmbda_1_Step_Divide10 (200, 256)
Gittins_Lmbda_1_Step_Divide100 (200, 256)
Gittins_Lmbda_0001_Step_Divide2 (200, 256)
Gittins_Lmbda_0001_Step_Divide10 (200, 256)
Gittins_Lmbda_0001_Step_Divide100 (200, 256)
